In [3]:
#extract coordinates from gadm40_IND_3.kml file. structure of extracted data should be 
#location={'distName':Polygon([(lat1,long1),(lat2,long2),...])}
#extract point coord from https://raw.githubusercontent.com/undpindia/Data4Policy/main/Geospatial%20Data%20Science%20Internship/Datasets/telangana_fires.csv
#Store them as {fID:{'year':y,'lat'=latf,'long':longf},....}
#compare whether points from fire dictionary falls inside polygons from 
#conditional statements to ensure genuine records
#store extracted info as {Rec1:{'adm_name':a,'year':y,'fireCount':C}}
#dumps into json file
import csv, urllib.request, xml.etree.ElementTree as ET, json
from pykml import parser
from shapely.geometry import Point,Polygon

#checking the structure of kml file
#fhand=open('gadm40_IND_3.kml')
#for line in fhand:
#    print(line)

fireRawData=urllib.request.urlopen('https://raw.githubusercontent.com/undpindia/Data4Policy/main/Geospatial%20Data%20Science%20Internship/Datasets/telangana_fires.csv')
fireData=[]
for line in fireRawData:
    fireData.append(line.decode())
    
#print(fireData)
header=[]   
fireParse=csv.reader(fireData)
header=next(fireParse)

#print(header)
count=0
fireCoord=[]
for row in fireParse:
    fireCoord.append(row)
    count=count+1
#print(count)    
#checking the code for number of records. number of fire events from 2015 to 2021 = 18785
fireDict={}
#fireDict contains the 'event number':{'date':d,'lat':latitude,'long':longitude}
#genuine record count is j
j=1
for i in range(count):
    fireDict[fireCoord[i][0]]={'date':fireCoord[i][6],'lat':float(fireCoord[i][1]),'long':float(fireCoord[i][2])}
#checking the structure and records in the disctionary    
#print(len(fireDict.keys())) 
#result=18785. Hence, fireDict has genuine records.
#print(fireDict)

telData=ET.parse('gadm40_IND_3.kml')
telDataRoot=telData.getroot()
#check the access to the structure of the kml file with indices
#print(telDataRoot[0][1][2000][3][0][0].text)

#grouping coordinate points
location={}
#location is a disctionary that will store 'adm_name':<polygons>
telPolygon=[]
#telPolygon refers to the polygons inside Telengana. Contains the list of polygons for each adm_boundary inside Telengana
lst=[]
#lst is a temporary storage list for the administration boundary coordinates
polygonCount=0
for i in range(2335):
    if telDataRoot[0][1][i+1][3][0][0].text!='Telangana':
        continue
    else:
        #location[telDataRoot[0][1][i+1][3][0][5].text]=telDataRoot[0][1][i][4][0][0][0][0].text.split()
        lst=telDataRoot[0][1][i+1][4][0][0][0][0].text.split()
        temp=[]
        
        for k in range(len(lst)):
            temp.append(lst[k].split(','))
            for j in range(2):
                temp[k][j]=float(temp[k][j])
        
        telPolygon.append(Polygon(temp))
        
        location[telDataRoot[0][1][i+1][3][0][5].text]=telPolygon[polygonCount]
        polygonCount=polygonCount+1

#print(i)            
#print(location)
#k=administration name, v=list of coordinates       
      
#for x in telDataRoot[0][1][1][4][0][0][0][0].text

#short for fire record
fRec={}

#counter for genuine number of records
j=1
for k,v in fireDict.items():
    #y=year
    y='20'+fireDict[k]['date'][6:8]
    x=str(j)
    coord=[fireDict[k]['long'],fireDict[k]['lat']]
    p=Point(coord)
    for a,poly in location.items():
        if p.within(poly):
            flag=0
            if j==1:
                fRec['rec'+x]={'adm_name':a,'year':y,'fireCount':1}
                j=j+1
                break
            else:
                for k,v in fRec.items():
                    if a==fRec[k]['adm_name'] and fRec[k]['year']==y:
                        fRec[k]['fireCount']=fRec[k]['fireCount']+1
                        flag=1
                        break
                if flag==0:
                    fRec['rec'+x]={'adm_name':a,'year':y,'fireCount':1}
                    j=j+1
            break
            #fireCount[k2]=fireCount.get(k2,0)+1
#check if all fire records are a part of the fRec dictionary. Result was True
#fCount=0
#for k,v in fRec.items():
#    fCount=fCount+fRec[k]['fireCount']
#print(len(fireDict.keys())==fCount)
#print(fRec)

#json output
fRecJSON=json.dumps(fRec,indent=4)
with open('output1.json','w') as f:
     f.write(fRecJSON)